In [1]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
from sklearn.datasets import make_blobs
import random
import json
import time
from tabulate import tabulate

from kdtree import *
from helper import *

In [2]:
savefile = "mst_data2.json"
to_plot = False
no_points = 10000
no_centres = 10

In [3]:
df = pd.read_csv("worldcitiespop.csv")
india_cities = df[df["Country"] == "in"]
print(india_cities.shape)
random_cities = india_cities.sample(n=no_points, random_state=42)

unique_cities = random_cities.drop_duplicates(subset=["Latitude", "Longitude"])

points = list(zip(unique_cities["Latitude"], unique_cities["Longitude"]))


print(len(points), "points", points[:10])

/tmp/ipykernel_51002/4163954900.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("worldcitiespop.csv")


(39813, 7)
9591 points [(17.063333, 78.654444), (28.971111, 74.833889), (24.583333, 73.716667), (20.15, 75.533333), (23.55, 74.45), (17.25, 80.15), (21.766667, 72.15), (25.833333, 73.483333), (26.6475, 78.88555600000001), (29.881389, 77.238333)]


In [4]:
# X, Y = make_blobs(n_samples=no_points, centers=no_centres, random_state=42)
# points = [(x, y) for x, y in X]

In [5]:
cordmap = {point: i for i, point in enumerate(points)}

In [6]:
def dcran(cordmap):
    tree = KDTree()
    tree.root = tree.build(points)
    
    G = nx.Graph()
    for coord, index in cordmap.items():
        G.add_node(index, pos=coord)
    k = 2
    ccount = len(cordmap)
    disset , visited = set() , set()
    
    while ccount != 1:
        dis2 = min(len(cordmap) , math.factorial(k) )
        disset.update([k , dis2])
        disset = disset - visited
        visited.update([k , dis2])
        
        for kdis in sorted(list(disset)):
            for pointi, i in cordmap.items():
                pointj = ith_nearest_neighbor(tree, pointi, kdis)
                dis = euclidean_distance(pointi, pointj)
                G.add_edge(i, cordmap[pointj], weight=dis)
            graphify(G, to_plot )
            ccount = count_clusters(G)
            print(kdis, ccount)
        k += 1
        
        if ccount == 1:
            break
        
    return G , tree

In [7]:
dcran_start_time = time.time()
kc , tree = dcran(cordmap)

2 2907
3 437
6 11
4 9
24 2
5 2
120 1


In [8]:
graphify(kc, to_plot )

In [9]:
G = kc
num_nodes = G.number_of_nodes()
conedge = num_edges = G.number_of_edges()

# Calculate the sum of all edge weights
total_weight = sum(data["weight"] for u, v, data in G.edges(data=True))

print(f"Total number of nodes: {num_nodes}")
print(f"Total number of edges: {num_edges}")
print(f"Total sum of edge weights: {total_weight}")

Total number of nodes: 9591
Total number of edges: 48138
Total sum of edge weights: 17465.048806714298


In [10]:

def plot_kd_tree(node, min_x, max_x, min_y, max_y, prev_node, branch, depth=0):
    """Plot the kd-tree using matplotlib."""
    if node is None:
        return
    
    cur_point = node.point
    axis = node.axis

    if axis == 0:
        if branch is not None and prev_node is not None:
            if branch:
                max_y = prev_node.point[1]
            else:
                min_y = prev_node.point[1]
                
        plt.plot([cur_point[0], cur_point[0]], [min_y, max_y], 'r')
        plot_kd_tree(node.left, min_x, cur_point[0], min_y, max_y, node, True, depth + 1)
        plot_kd_tree(node.right, cur_point[0], max_x, min_y, max_y, node, False, depth + 1)

    elif axis == 1:
        if branch is not None and prev_node is not None:
            if branch:
                max_x = prev_node.point[0]
            else:
                min_x = prev_node.point[0]
                
        plt.plot([min_x, max_x], [cur_point[1], cur_point[1]], 'b')
        plot_kd_tree(node.left, min_x, max_x, min_y, cur_point[1], node, True, depth + 1)
        plot_kd_tree(node.right, min_x, max_x, cur_point[1], max_y, node, False, depth + 1)

    plt.plot(cur_point[0], cur_point[1], 'ko')

# Plotting


In [11]:
mst = nx.minimum_spanning_tree(G, algorithm="prim", weight="weight")
graphify(mst, to_plot )



In [12]:
G = mst
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()


foundw = total_weight = sum(data["weight"] for u, v, data in G.edges(data=True))

print(f"Total number of nodes: {num_nodes}")
print(f"Total number of edges: {num_edges}")
print(f"Total sum of edge weights: {total_weight}")
dcran_end_time = time.time()
dcran_elapsed_time = dcran_end_time - dcran_start_time

Total number of nodes: 9591
Total number of edges: 9590
Total sum of edge weights: 918.5472443277376


In [13]:
stmst_start_time = time.time()
G = nx.Graph()

for pointi, i in cordmap.items():
    G.add_node(i, pos=pointi)
    for pointj, j in cordmap.items():
        if i != j:
            dis = euclidean_distance(pointi, pointj)
            G.add_edge(i, j, weight=dis)

In [14]:
mst = nx.minimum_spanning_tree(G, algorithm="prim", weight="weight")
graphify(mst, to_plot  )

In [15]:
G = mst
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

# Calculate the sum of all edge weights
realw = total_weight = sum(data["weight"] for u, v, data in G.edges(data=True))

print(f"Total number of nodes: {num_nodes}")
print(f"Total number of edges: {num_edges}")
print(f"Total sum of edge weights: {total_weight}")
stmst_end_time = time.time()
stmst_elapsed_time = stmst_end_time - stmst_start_time

Total number of nodes: 9591
Total number of edges: 9590
Total sum of edge weights: 914.3557313916614


In [16]:
percentage_error = ((foundw - realw) / realw) * 100

# Format the output for readability
formatted_output = f"Real Weight: {realw}  Found Weight: {foundw}  no of Edge: {conedge} Percentage Error: {percentage_error:.2f}%"
formatted_output

'Real Weight: 914.3557313916614  Found Weight: 918.5472443277376  no of Edge: 48138 Percentage Error: 0.46%'

In [17]:
with open(savefile, "r") as f:
    loaded_data = json.load(f)
print(loaded_data)
currres = []
speedup = round((stmst_elapsed_time  / dcran_elapsed_time) , 2)
loaded_data.append(
    [
        no_points,
        no_centres,
        foundw,
        realw,
        conedge,
        100 - percentage_error ,
        dcran_elapsed_time ,
        stmst_elapsed_time ,
        speedup
    ]
)

# Save the updated dictionary back to the JSON file
with open(savefile, "w") as f:
    json.dump(loaded_data, f)

[[1000, 10, 262.4419314705513, 258.819134723451, 8252, 98.6, 1.83, 4.38, 2.4], [10000, 10, 866.0209239473647, 865.3329904388614, 90645, 99.921, 28.58, 551.35, 19.29], [10000, 100, 1342.6324139702915, 1337.4478711909765, 27591, 99.612, 3.24, 510.53, 157.39], [10000, 100, 1342.6324139702915, 1337.4478711909762, 27591, 99.612, 2.48, 632.89, 255.22], [10000, 1000, 1459.7289235192638, 1455.1233116849917, 27457, 99.683, 2.5, 565.44, 226.17], [10000, 10, 866.0209239473647, 865.3329904388614, 90645, 99.92050071867081, 24.179672241210938, 617.7955191135406, 25.55], [10000, 50, 1260.1226131964793, 1255.8337705131123, 27661, 99.65848643474409, 2.1823902130126953, 513.3643357753754, 235.23], [10000, 10, 866.0209239473647, 865.3329904388614, 90645, 99.92050071867081, 24.11609435081482, 543.8352909088135, 22.55], [10000, 10, 875.7172119877604, 865.3329904388614, 38587, 98.79997392175785, 16.157432556152344, 646.0387144088745, 39.98], [10000, 100, 1342.6324139702872, 1337.4478711909717, 27591, 99.612

In [18]:
with open(savefile, "r") as f:
    loaded_data = json.load(f)
print(loaded_data)

[[1000, 10, 262.4419314705513, 258.819134723451, 8252, 98.6, 1.83, 4.38, 2.4], [10000, 10, 866.0209239473647, 865.3329904388614, 90645, 99.921, 28.58, 551.35, 19.29], [10000, 100, 1342.6324139702915, 1337.4478711909765, 27591, 99.612, 3.24, 510.53, 157.39], [10000, 100, 1342.6324139702915, 1337.4478711909762, 27591, 99.612, 2.48, 632.89, 255.22], [10000, 1000, 1459.7289235192638, 1455.1233116849917, 27457, 99.683, 2.5, 565.44, 226.17], [10000, 10, 866.0209239473647, 865.3329904388614, 90645, 99.92050071867081, 24.179672241210938, 617.7955191135406, 25.55], [10000, 50, 1260.1226131964793, 1255.8337705131123, 27661, 99.65848643474409, 2.1823902130126953, 513.3643357753754, 235.23], [10000, 10, 866.0209239473647, 865.3329904388614, 90645, 99.92050071867081, 24.11609435081482, 543.8352909088135, 22.55], [10000, 10, 875.7172119877604, 865.3329904388614, 38587, 98.79997392175785, 16.157432556152344, 646.0387144088745, 39.98], [10000, 100, 1342.6324139702872, 1337.4478711909717, 27591, 99.612

In [19]:
headers = [
    "Points",
    "Centres",
    "DCRAN Wt",
    "kruskal Wt",
    "Edge count",
    "Acc(%)",
    "DCRAN Time (s)",
    "STMST Time (s)",
    "Speedup"
]

# Format the data as a table using tabulate
table_str = tabulate(loaded_data[-10:], headers, tablefmt="pipe", floatfmt=(".0f", ".0f", ".1f", ".1f", ".0f", ".2f", ".2f", ".2f", ".2f"))
print(table_str)




|   Points |   Centres |   DCRAN Wt |   kruskal Wt |   Edge count |   Acc(%) |   DCRAN Time (s) |   STMST Time (s) |   Speedup |
|---------:|----------:|-----------:|-------------:|-------------:|---------:|-----------------:|-----------------:|----------:|
|      117 |         1 |       18.0 |         17.6 |          336 |    97.58 |             0.14 |             0.04 |      0.32 |
|    10000 |         3 |      157.2 |        155.3 |        28445 |    98.75 |             3.00 |           589.76 |    196.78 |
|     8865 |         3 |      904.3 |        879.8 |        34288 |    97.21 |            10.64 |           576.56 |     54.17 |
|      998 |         3 |      324.1 |        315.8 |         2923 |    97.38 |             0.79 |            11.79 |     14.94 |
|      998 |         3 |      324.1 |        315.8 |         2923 |    97.38 |             0.31 |             4.42 |     14.35 |
|      998 |         3 |      324.1 |        315.8 |         2923 |    97.38 |             0.30 |